# Reimplementation of original kBET
- kBET on the largest component of Sc_04 cell type 2
- run with R(kbet)

In [1]:
#truncated normal distribution distribution function
ptnorm <- function(x, mu, sd, a=0, b=1, alpha = 0.05, verbose = FALSE){
  #this is the cumulative density of the truncated normal distribution
  #x ~ N(mu, sd^2), but we condition on a <= x <= b
  if (!is.na(x)){


    if (a > b) {
      warning("Lower and upper bound are interchanged.")
      tmp <- a
      a <- b
      b <- tmp
    }

    if (sd <= 0 || is.na(sd)) {
      if (verbose) {
        warning("Standard deviation must be positive.")
      }
      if (alpha <= 0) {
        stop("False positive rate alpha must be positive.")
      }
      sd <- alpha
    }
    if (x < a || x > b) {
      warning("x out of bounds.")
      cdf <- as.numeric(x > a)
    } else {
      alp <- pnorm((a - mu) / sd)
      bet <- pnorm((b - mu) / sd)
      zet <- pnorm((x - mu) / sd)
      cdf <- (zet - alp) / (bet - alp)
    }
    cdf
  } else {
    return(NA)
  }
}

In [2]:
# residual score function of kBET
residual_score_batch <- function(knn.set, class.freq, batch) {
    # knn.set: indices of nearest neighbors
    # class.freq: global batch proportions
    # batch: batch labels

    # return NA if all values of neighborhood are NA (which may arise from subsampling a knn-graph)
    if (all(is.na(knn.set))) {
        return(NA)
    }
    else{
        # extracts the batch labels of all neighbors (excluding NA)
        # computes local batch frequencies (observed)
        freq.env <- table(batch[knn.set[!is.na(knn.set)]]) / length(!is.na(knn.set))
        
        # create zero vector with length of batches and initialize it with freqs of batches
        full.classes <- rep(0, length(class.freq$class))
        full.classes[class.freq$class %in% names(freq.env)] <- freq.env
        
        # global batch props (expected)
        exp.freqs <- class.freq$freq

        # compute chi-square test statistics
        ## sum((observed - expected)^2 / expected)
        chi_sq_statistic <- sum((full.classes - exp.freqs)^2 / exp.freqs)
        
        return(chi_sq_statistic)
    }
}

In [3]:
# core function of kBET; called for each sample
chi_batch_test <- function(knn.set, class.freq, batch, df) {
    # knn.set: indices of nearest neighbors
    # class.freq: global batch proportions
    # batch: batch labels
    # df: degrees of freedom

    # return NA if all values of neighborhood are NA (which may arise from subsampling a knn-graph)
    if (all(is.na(knn.set))) {
        return(NA)
    }
    else {
        # extracts the batch labels of all neighbors (excluding NA)
        # computes local batch counts (observed)
        freq.env <- table(batch[knn.set[!is.na(knn.set)]])

        # create zero vector with length of batches and initialize it with counts of batches
        full.classes <- rep(0, length(class.freq$class))
        full.classes[class.freq$class %in% names(freq.env)] <- freq.env

        # global batch counts (for this sample)
        exp.freqs <- class.freq$freq * length(knn.set)

        # compute chi-square test statistics
        ## sum((observed - expected)^2 / expected)
        chi.sq.value <- sum((full.classes - exp.freqs)^2 / exp.freqs)

        # calculate p-value
        result <- 1 - pchisq(chi.sq.value, df)

        # I actually would like to know when 'NA' arises.
        if (is.na(result)) {
            return(NA)
        }
        else {
           return(result)
        }
    }
}

In [4]:
# df: dataset (rows: cells, columns: genes)
# batch: batch labels for each cell
# k0 = NULL: # neighbors to test on
# knn = NULL: n_cells x k0 matrix of kNN indices (optional)
# testSize = NULL: # data points to test (default is 10% of entire dataset, but at least 25)
# do.pca = TRUE: perform PCA before kNN search
# dim.pca = 50: if do.pca=TRUE then # dims
# heuristic = TRUE: compute an optimal neighborhood size k
# n_repeat = 100: create statistics on batch estimates running test on n_repeat subsets
# alpha = 0.05: significance level
# addTest = FALSE: perform Likelihood Ratio Test approximation to the multinimial test and
#                a multinomial exact test (if appropriate)
# verbose = FALSE: display stages of current computation
# plot = TRUE: if stats > 10 then a boxplot of the resulting rejection rates is created
#adapt = TRUE: in some cases a number of cells do not contribute to any neighborhood and 
#              this may cause an imbalance in observed and expected batch label frequencies.
#              Frequencies will be adapted if adapt=TRUE.


In [5]:
# source("kBET-utils.R")

In [6]:
library(ggplot2)

In [7]:
sc_dir <- "2/largest_comp/"
sc_dir

[1] "2/largest_comp/"

In [8]:
df <- as.matrix(read.csv(paste0(sc_dir, "df.csv"), header=FALSE))
batch <- factor(read.csv(paste0(sc_dir, "batch.csv"), header = FALSE)[[1]])
k0 <- 70
knn <- as.matrix(read.csv(paste0(sc_dir, "knn.csv"), header = FALSE))
testSize = NULL
do.pca = FALSE
dim.pca = 50
heuristic = FALSE
n_repeat = 100
alpha = 0.05
addTest = FALSE
verbose = TRUE
plot = TRUE
adapt = FALSE

dim(df)
length(batch)
k0
dim(knn)
testSize
do.pca
dim.pca
heuristic
n_repeat
alpha
addTest
verbose
plot
adapt

[1] 600  70

[1] 600

[1] 70

[1] 600  70

NULL

...

[1] 0.05

[1] FALSE

[1] TRUE

[1] TRUE

[1] FALSE

In [9]:
length(unique(batch))

[1] 2

In [10]:
is.factor(batch)

[1] TRUE

In [11]:
dof <- length(unique(batch)) - 1    # degrees of freedom

dof

[1] 1

In [12]:
if (is.factor(batch)) {
    batch <- droplevels(batch)
}

batch

[1] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [10] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [19] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [28] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [37] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [46] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [55] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [64] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [73] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [82] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [91] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[100] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[109] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[118] batch 1 batch 1 batch 1 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[127] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[136] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[145] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[154] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[163] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[172] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[181] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[190] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[199] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[208] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[217] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[226] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[235] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[244] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[253] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[262] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[271] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[280] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[289] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[298] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[307] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[316] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[325] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[334] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[343] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[352] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[361] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[370] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[379] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[388] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[397] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[406] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[415] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[424] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[433] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[442] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[451] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[460] batch 2 batch 2 ba

In [13]:
table(batch)
length(batch)

batch
batch 1 batch 2 
    120     480 

[1] 600

In [14]:
frequencies <- table(batch) / length(batch)

frequencies

batch
batch 1 batch 2 
    0.2     0.8 

In [15]:
# get 3 different permutations of the batch label
batch.shuff <- replicate(3, batch[sample.int(length(batch))])

head(batch.shuff)

batch 2,batch 2,batch 2
batch 1,batch 2,batch 1
batch 2,batch 2,batch 2
batch 2,batch 2,batch 2
batch 2,batch 2,batch 2
batch 1,batch 2,batch 1


In [16]:
dim(batch.shuff)

[1] 600   3

In [17]:
class.frequency <- data.frame(
    class = names(frequencies),
    freq = as.numeric(frequencies)
)

class.frequency

class,freq
<chr>,<dbl>
batch 1,0.2
batch 2,0.8


In [18]:
dataset <- df

dim(dataset)

[1] 600  70

In [19]:
dim.dataset <- dim(dataset)

dim.dataset

[1] 600  70

In [20]:
dim.dataset[1]
dim.dataset[2]

[1] 600

[1] 70

In [21]:
# check dimensions
if (dim.dataset[1] != length(batch) && dim.dataset[2] != length(batch)) {
    stop("Input matrix and batch information do not match. Execution halted.")
}

In [22]:
if (dim.dataset[2] == length(batch) && dim.dataset[1] != length(batch)) {
    if (verbose) {
      cat('Input matrix has samples as columns. kBET needs samples as rows. Transposing...\n')
    }
    dataset <- t(dataset)
    dim.dataset <- dim(dataset)
  }

In [23]:
# check if the dataset is too small
if (dim.dataset[1] <= 10) {
    if (verbose) {
        cat("Your dataset has less than 10 samples. Abort and return NA.\n")
    }
    return(NA)
}

In [24]:
stopifnot(class(n_repeat) == 'numeric', n_repeat > 0)

In [25]:
# if k0 was set by the user and is too small & we do not operate on a knn graph, abort

# the reason is that if we want to test kBET on knn graph data integration methods,
# we usually face small numbers of nearest neighbours.
if (k0 < 10 & is.null(knn))
{
    if (verbose)
    {
        warning(
            "Your dataset has too few samples to run a heuristic.\n",
            "Return NA.\n",
            "Please assign k0 and set heuristic=FALSE."
        )
    }
    return(NA)
}

In [26]:
# set # tests
if (is.null(testSize) || (floor(testSize) < 1 || dim.dataset[1] < testSize))
{
    test.frac <- 0.10   # 10% of datapoints
    testSize <- ceiling(dim.dataset[1] * test.frac)
    
    if (testSize < 2 && dim.dataset[1] > 25)
    {
        testSize <- 25
    }

    if (verbose)
    {
        cat('Number of kBET tests is set to ')
        cat(paste0(testSize, '.\n'))
    }
}

Number of kBET tests is set to 60.


In [27]:
# result list
rejection <- list()
rejection$summary <- data.frame(
    kBET.expected = numeric(4),
    kBET.observed = numeric(4),
    kBET.signif = numeric(4)
)

rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0


In [28]:
rejection$results <- data.frame(
    tested = numeric(dim.dataset[1]),
    kBET.pvalue.test = rep(0, dim.dataset[1]),
    kBET.pvalue.null = rep(0, dim.dataset[1])
)

rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0


In [29]:
knn

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [30]:
# concat knn matrix so
## 1st col comes 1st then 2nd col ... then k0-1 th col then cells itself
env <- as.vector(cbind(knn[, seq_len(k0 - 1)], seq_len(dim.dataset[1])))
env

[1]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
   [19]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
   [37]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
   [55]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
   [73]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
   [91]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [109]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA 196  34 313  37 443   5
  [127] 329 224 369 226  20  74 189 326  19 439 242 185 292 111 356 256  20 222
  [145] 346 361 360  17 361 369 439 349  24 454 278 188 465  85 181 220  53 214
  [163]  25 134 162 390   1 332 334 393 236 435 454  92 188 323 384 247 141 285
  [181]  18  25 198 228 371 257 360 327 222 405 314 210 368 134 189 236 384  89
  [199] 265  66  41 276 289 228 448 436 226 297 441 236 454 353 318 320 448 298
  [217]  39 330 139 443 113  32 236 218  92 368  25  54 328 419 108 308   5 280
  [235] 311 340 187 146 470  33 461 101  85 439 228 368 228 251 111 458 111 274
  [253] 303 272 376  66 395 134 261 341 236 147 436 222 303 368 406  62  59 236
  [271]  91 229 169 315 169 422  89 276 208 313 448 276 371 175 213 282 214 423
  [289] 396 195  36 431 376 346 439 141  69 455 112 343  81 224 410 248  57 221
  [307] 151 213  85 292 380  47 416 385 463  47 110  86 223 229 238 228  22 274
  [325] 451 129  93 169 463  28 470 189 100 369 222 311 236 208 454 104  25 380
  [343] 109 150 384 161 157 398 232  32 292 114 436 331 193 410 304 119 210 284
  [361] 155 252  51  19 436 171  12 338 386 188 423 285 433 285 402  81 192 189
  [379] 305 408 317   5 291 436 340 184 410 372   5 434  24 113 447  87 354 246
  [397] 412   5 208 312 448 436  93 210 374 313 331 360 228  12 328 145 436 276
  [415]  77 369 240 292 402 240  28 443 221 443 436 357 334  99 110 292 403 368
  [433] 331 352 138  47 380 312 210 144 431 354 304 231  24 475 217 232  80  98
  [451] 427 452 311 406 186 305  52 352 179 368 191 341  73 338 180 214 416  93
  [469] 228 395  85 214 281 439 396  46  79  13 334 443 218 253  23 214   5 292
  [487] 245 478 261  94   5 414 128  81 158 317 111 146 189 144  93 198 274 224
  [505] 471 353 155  32 105 290  47 310 247 439 232 443 475 367 270 189  17 354
  [523] 237 368 328 299 310 339 361 308  76  31 240 111 326  26 141 477 179 248
  [541] 150 257  16  47 219 292 435 250 135 194  91 238 240 304 161 326 398 420
  [559]  15  17 248 307 219 449 137 146 398 413 119 350 478 464 181 314  93 326
  [577]  15 116 134 275   5 110 167 447 369  93 367 298 174 130 326 229 223 433
  [595] 298 315 299  47   9 413  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [613]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [631]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [649]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [667]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [685]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [703]  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA  NA
  [721] 273 228 405   6 127 134 292  39 339 178 412 436 434  38  76  73 406 220
  [739] 311 214   5 463 443 292 282  81 214  95 459 426 461 292 218 333 417   1
  [757]  20 228 210 428 344 120 110 125 318 253 369 419 443 225 330 146   8 368
  [775]  85 222 196 222 380 360 228 128 122 439   5 442 448 108  81  31  77 292
  [793] 274 436  15 354 301  97 415 233 362 285 245  35  53 274  15 201 240  54
  [811] 463 205  14 297 310 480 197 119 416 295 301 460 298  93 356 354 189 102
  [829]   5   5 447 168 189 292 287 401 255 243 401 468 274 313 274 359 447  79
  [847] 274 447 439 307 214 187 365 204  94  80 205 283 146 479  87 376 179 201
  [865] 229 122 185 166 443 298 334 193  47 148 248 436  93 436 214  41 455 208
  [883] 274 215 134 322 272 410 119 114 416 102 477 189 111 362 383  81 283 443
  [9

In [31]:
length(env)

[1] 42000

In [32]:
cf <- if (adapt && is.imbalanced) new.class.frequency else class.frequency
cf

class,freq
<chr>,<dbl>
batch 1,0.2
batch 2,0.8


In [33]:
length(batch)

[1] 600

In [34]:
# get avg p-value, p = 1 - CDF
rejection$average.pval <- 1 - pchisq(k0 * residual_score_batch(env, cf, batch), dof)
rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0


In [35]:
# initialize intermediates
kBET.expected <- numeric(n_repeat)
kBET.observed <- numeric(n_repeat)
kBET.signif <- numeric(n_repeat)

length(kBET.expected)
kBET.expected

[1] 100

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [36]:
testSize

[1] 60

In [37]:
alpha

[1] 0.05

In [38]:
# kBET; run n_repeat = 100 times
for (i in seq_len(n_repeat))
{
    # choose a random sample from dataset (rows: cells, cols: genes)
    # size of random sample is 10% of dataset by default
    idx.runs <- sample.int(dim.dataset[1], size=testSize)
    # print(idx.runs)
    
    # get the neighbors of sample cells; also attach sample cells as self-neighbor
    env <- cbind(knn[idx.runs, seq_len(k0 - 1)], idx.runs)
    # print(env)

    # perform test for each cell in sample
    cf <- if (adapt && is.imbalanced) new.class.frequency else class.frequency  # global batch props
    # apply chi_batch_test to each row of env matrix; 1 is rows
    p.val.test <- apply(env, 1, chi_batch_test, cf, batch, dof)
    # print(p.val.test)

    is.rejected <- p.val.test < alpha

    # apply the same test but this time batch labels are permuted. 
    # permutations are columns of batch.shuff; 2 is columns
    p.val.test.null <- apply(
        batch.shuff, 2,
        function(x) apply(env, 1, chi_batch_test, class.frequency, x, dof)
    )

    # expected rejection rate under the null for this run
    # for each shuffle compute the fraction of cells with p < alpha, then average across shuffles
    kBET.expected[i] <- mean(
        apply(
            p.val.test.null, 2,
            function(x) sum(x < alpha, na.rm = TRUE) / sum(!is.na(x))
        )
    )

    # observed rejection rate in this run
    # fraction of tested cells rejecting the null with the real labels
    kBET.observed[i] <- sum(is.rejected, na.rm = TRUE) / sum(!is.na(p.val.test))

    # compute significance
    kBET.signif[i] <- 1 - ptnorm(
        kBET.observed[i],
        mu = kBET.expected[i],
        sd = sqrt(kBET.expected[i] * (1 - kBET.expected[i]) / testSize),
        alpha = alpha
    )

    # mark which cells were sampled in this run and store
    ## their observed per-cell p-values
    ## their mean null per-cell p-values (averaged over shuffles)
    rejection$results$tested[idx.runs] <- 1
    rejection$results$kBET.pvalue.test[idx.runs] <- p.val.test
    rejection$results$kBET.pvalue.null[idx.runs] <- rowMeans(p.val.test.null, na.rm=TRUE)
}

In [39]:
rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
1,1.110223e-16,1.110223e-16
1,1.110223e-16,1.110223e-16
1,1.110223e-16,1.110223e-16


In [40]:
kBET.observed

[1] 0.1333333 0.3000000 0.2833333 0.2833333 0.3500000 0.2500000 0.2500000
  [8] 0.2500000 0.1333333 0.2500000 0.1833333 0.3333333 0.3333333 0.2833333
 [15] 0.2500000 0.3166667 0.3166667 0.2833333 0.2666667 0.2666667 0.3500000
 [22] 0.2833333 0.4166667 0.3166667 0.2666667 0.1833333 0.1333333 0.2166667
 [29] 0.2500000 0.2500000 0.3833333 0.2333333 0.2500000 0.2666667 0.3166667
 [36] 0.2500000 0.2833333 0.2666667 0.3000000 0.3500000 0.2666667 0.3333333
 [43] 0.3333333 0.2000000 0.2000000 0.2666667 0.2666667 0.3000000 0.2500000
 [50] 0.2166667 0.2833333 0.1500000 0.2666667 0.1833333 0.2333333 0.4166667
 [57] 0.2666667 0.1500000 0.3166667 0.3333333 0.2666667 0.2666667 0.3000000
 [64] 0.2333333 0.3333333 0.2333333 0.2500000 0.2500000 0.2833333 0.2500000
 [71] 0.2833333 0.2666667 0.3166667 0.2666667 0.2833333 0.2500000 0.1833333
 [78] 0.2166667 0.2000000 0.2666667 0.2333333 0.3333333 0.2166667 0.2666667
 [85] 0.1666667 0.1833333 0.2666667 0.2333333 0.2000000 0.2500000 0.2833333
 [92] 0.1333333 0.3000000 0.2333333 0.2666667 0.1833333 0.2166667 0.3500000
 [99] 0.2666667 0.3000000

In [41]:
length(kBET.observed)

[1] 100

In [42]:
sc_dir

[1] "2/largest_comp/"

In [43]:
# summarize exptected, observed and run-level significance across runs
if (n_repeat > 1)
{
    CI95 <- c(0.025, 0.5, 0.975)

    rejection$summary$kBET.expected <- c(
        mean(kBET.expected, na.rm=TRUE),
        quantile(kBET.expected, CI95, na.rm=TRUE)
    )

    rownames(rejection$summary) <- c('mean', '2.5%', '50%', '97.5%')

    rejection$summary$kBET.observed <- c(
        mean(kBET.observed, na.rm=TRUE),
        quantile(kBET.observed, CI95, na.rm=TRUE)
    )

    rejection$summary$kBET.signif <- c(
        mean(kBET.signif, na.rm=TRUE),
        quantile(kBET.signif, CI95, na.rm=TRUE)
    )

    # also return per run vectors
    rejection$stats$kBET.expected <- kBET.expected
    rejection$stats$kBET.observed <- kBET.observed
    rejection$stats$kBET.signif <- kBET.signif

    if (n_repeat < 10)
    {
        cat('Warning: The quantile computation for ')
        cat(paste0(n_repeat))
        cat(' subset results is not meaningful.')
    }

    if (plot)
    {
        plot.data <- data.frame(
            class = rep(c('observed(kBET)', 'expected(random)'), each=n_repeat),
            data = c(kBET.observed, kBET.expected)
        )

        g <- ggplot(plot.data, aes(class, data)) + geom_boxplot() + theme_bw() + labs(x = 'Test', y = 'Rejection rate') + scale_y_continuous(limits = c(0, 1))
        # print(g)
        ggsave(paste0(sc_dir, "kBET_boxplot.pdf"), plot = g, width = 7, height = 5, dpi = 300)
    }
} else
{
    rejection$summary$kBET.expected <- kBET.expected[1]
    rejection$summary$kBET.observed <- kBET.observed[1]
    rejection$summary$kBET.signif <- kBET.signif[1]
}

In [44]:
# collect parameters
rejection$params <- list()
rejection$params$k0 <- k0
rejection$params$testSize <- testSize
rejection$params$do.pca <- do.pca
rejection$params$dim.pca <- dim.pca
rejection$params$heuristic <- heuristic
rejection$params$n_repeat <- n_repeat
rejection$params$alpha <- alpha
rejection$params$addTest <- addTest
rejection$params$verbose <- verbose
rejection$params$plot <- plot

In [45]:
rejection

$summary
      kBET.expected kBET.observed kBET.signif
mean      0.2423889     0.2620000  0.37437772
2.5%      0.1359722     0.1333333  0.07128421
50%       0.2416667     0.2666667  0.37523603
97.5%     0.3444444     0.3675000  0.75164865

$results
    tested kBET.pvalue.test kBET.pvalue.null
1        1     1.110223e-16     1.110223e-16
2        1     1.110223e-16     1.110223e-16
3        1     1.110223e-16     1.110223e-16
4        1     1.110223e-16     1.110223e-16
5        1     1.110223e-16     1.110223e-16
6        1     1.110223e-16     1.110223e-16
7        1     1.110223e-16     1.110223e-16
8        1     1.110223e-16     1.110223e-16
9        1     1.110223e-16     1.110223e-16
10       1     1.110223e-16     1.110223e-16
11       1     1.110223e-16     1.110223e-16
12       1     1.110223e-16     1.110223e-16
13       1     1.110223e-16     1.110223e-16
14       1     1.110223e-16     1.110223e-16
15       1     1.110223e-16     1.110223e-16
16       1     1.110223e-16     1.110223e-16
17       1     1.110223e-16     1.110223e-16
18       1     1.110223e-16     1.110223e-16
19       1     1.110223e-16     1.110223e-16
20       1     1.110223e-16     1.110223e-16
21       1     1.110223e-16     1.110223e-16
22       1     1.110223e-16     1.110223e-16
23       1     1.110223e-16     1.110223e-16
24       1     1.110223e-16     1.110223e-16
25       1     1.110223e-16     1.110223e-16
26       1     1.110223e-16     1.110223e-16
27       1     1.110223e-16     1.110223e-16
28       1     1.110223e-16     1.110223e-16
29       1     1.110223e-16     1.110223e-16
30       1     1.110223e-16     1.110223e-16
31       1     1.110223e-16     1.110223e-16
32       1     1.110223e-16     1.110223e-16
33       1     1.110223e-16     1.110223e-16
34       1     1.110223e-16     1.110223e-16
35       1     1.110223e-16     1.110223e-16
36       1     1.110223e-16     1.110223e-16
37       1     1.110223e-16     1.110223e-16
38       1     1.110223e-16     1.110223e-16
39       1     1.110223e-16     1.110223e-16
40       1     1.110223e-16     1.110223e-16
41       1     1.110223e-16     1.110223e-16
42       1     1.110223e-16     1.110223e-16
43       1     1.110223e-16     1.110223e-16
44       1     1.110223e-16     1.110223e-16
45       1     1.110223e-16     1.110223e-16
46       1     1.110223e-16     1.110223e-16
47       1     1.110223e-16     1.110223e-16
48       1     1.110223e-16     1.110223e-16
49       1     1.110223e-16     1.110223e-16
50       1     1.110223e-16     1.110223e-16
51       1     1.110223e-16     1.110223e-16
52       1     1.110223e-16     1.110223e-16
53       1     1.110223e-16     1.110223e-16
54       1     1.110223e-16     1.110223e-16
55       1     1.110223e-16     1.110223e-16
56       1     1.110223e-16     1.110223e-16
57       1     1.110223e-16     1.110223e-16
58       1     1.110223e-16     1.110223e-16
59       1     1.110223e-16     1.110223e-16
60       1     1.110223e-16     1.110223e-16
61       1     1.110223e-16     1.110223e-16
62       1     1.110223e-16     1.110223e-16
63       1     1.110223e-16     1.110223e-16
64       1     1.110223e-16     1.110223e-16
65       1     1.110223e-16     1.110223e-16
66       1     1.110223e-16     1.110223e-16
67       1     1.110223e-16     1.110223e-16
68       1     1.110223e-16     1.110223e-16
69       1     1.110223e-16     1.110223e-16
70       1     1.110223e-16     1.110223e-16
71       1     1.110223e-16     1.110223e-16
72       1     1.110223e-16     1.110223e-16
73       1     1.110223e-16     1.110223e-16
74       1     1.110223e-16     1.110223e-16
75       1     1.110223e-16     1.110223e-16
76       1     1.110223e-16     1.110223e-16
77       1     1.110223e-16     1.110223e-16
78       1     1.110223e-16     1.110223e-16
79       1     1.110223e-16     1.110223e-16
80       1     1.110223e-16     1.110223e-16
81       1     1.110223e-16     1.110223e-16
82       1     1.110223e-16     1.110223e-16
83       1     1

In [46]:
rejection$summary$kBET.observed

[1] 0.2620000 0.1333333 0.2666667 0.3675000

In [47]:
score <- rejection$summary$kBET.observed[1]

In [48]:
score

[1] 0.262

In [49]:
print("End of the notebook")

[1] "End of the notebook"
